In [1]:
!pip install sqlalchemy==1.3.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.8 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=a23f8da552ea1bbc9d2ee8119d57a047a13df9c6225a0e7972b3e71dc70fdaa5
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [10]:
#Display the names of the unique launch sites in the space mission
%sql select DISTINCT LAUNCH_SITE from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [11]:
#Display 5 records where launch sites begin with the string 'CCA'
%sql select * from SPACEXTABLE where launch_site like 'CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [12]:
#Display the total payload mass carried by boosters launched by NASA (CRS)
%sql select sum(payload_mass__kg_) as sum from SPACEXTABLE where customer like 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


sum
45596


In [13]:
#Display average payload mass carried by booster version F9 v1.1
%sql select avg(payload_mass__kg_) as Average from SPACEXTABLE where booster_version like 'F9 v1.1%'

 * sqlite:///my_data1.db
Done.


Average
2534.6666666666665


In [14]:
#List the date when the first succesful landing outcome in ground pad was acheived.
%sql select min(date) as Date from SPACEXTABLE where mission_outcome like 'Success'

 * sqlite:///my_data1.db
Done.


Date
2010-04-06


In [17]:
#List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql select booster_version from SPACEXTABLE where (Mission_Outcome like 'Success') AND (PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000) AND (Landing_Outcome like 'Success (drone ship)')

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [18]:
#List the total number of successful and failure mission outcomes
%sql SELECT mission_outcome, count(*) as Count FROM SPACEXTABLE GROUP by mission_outcome ORDER BY mission_outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [20]:
#List the names of the booster_versions which have carried the maximum payload mass. Use a subquery
maxm = %sql select max(payload_mass__kg_) from SPACEXTABLE maxv = maxm[0][0]
%sql select booster_version from SPACEXTABLE where payload_mass__kg_=(select max(payload_mass__kg_) from SPACEXTABLE)

 * sqlite:///my_data1.db
(sqlite3.OperationalError) near "=": syntax error
[SQL: select max(payload_mass__kg_) from SPACEXTABLE maxv = maxm[0][0]]
(Background on this error at: http://sqlalche.me/e/e3q8)
 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [23]:
#List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
%sql select date (DATE) as Month, landing_outcome, booster_version, launch_site from SPACEXTABLE where DATE like '2015%' AND landing_outcome like 'Failure (drone ship)'

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
2015-10-01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
2015-04-14,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [24]:
#Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.
%sql select landing_outcome, count(*) as count from SPACEXTABLE where Date >= '2010-06-04' AND Date <= '2017-03-20' GROUP by landing_outcome ORDER BY count Desc

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count
No attempt,10
Success (ground pad),5
Success (drone ship),5
Failure (drone ship),5
Controlled (ocean),3
Uncontrolled (ocean),2
Precluded (drone ship),1
Failure (parachute),1
